In [ ]:
import os
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import collections
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')
def RandomForestLDA(UserName,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'ArousalEncoded'+".csv")
    ArousalFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    file_path = os.path.join("ChannelsExtracted",UserName, 'ValenceEncoded'+".csv")
    ValenceFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test , z_train,z_test= train_test_split(BandsFile, ValenceFile,ArousalFile, test_size=0.2)
# Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)  	
# LDA Classifier Valence=y
    lda = LDA(n_components=20)  
    X_train = lda.fit_transform(X_train, y_train)  
    X_test = lda.transform(X_test)  
    classifier = RandomForestClassifier(max_depth=2, random_state=0)
    classifier.fit(X_train, y_train)  
    y_predict = classifier.predict(X_test)
# LDA Classifier Arousal=z
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    lda = LDA(n_components=20)  
    X_train = lda.fit_transform(X_train, z_train)  
    X_test = lda.transform(X_test)  
    classifier = RandomForestClassifier(max_depth=2, random_state=0)
    classifier.fit(X_train, z_train)  
    z_predict = classifier.predict(X_test)
    cm = confusion_matrix(z_test, z_predict)
    #return f1 score and accurecies for valence and arousal
    return [accuracy_score(z_test, z_predict)*100,accuracy_score(y_test, y_predict)*100
            ,f1_score(z_test, z_predict, average='weighted'),
            f1_score(y_test, y_predict, average='weighted')]

############### Happy And Sad ####################
def RandomForestHS(UserName,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'HappyAndSadEncoded'+".csv")
    SurveyFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'HS'+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
    while ((np.sum(y_train == 0) == 0) or (np.sum(y_train == 1) == 0)): #Don't Start training unless you have 2 classes.. happens with imbalance
        print("here")
        X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
# Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)  	
# LDA Classifier Valence=y
    lda = LDA(n_components=20)  
    X_train = lda.fit_transform(X_train, y_train)  
    X_test = lda.transform(X_test)  
    classifier = RandomForestClassifier(max_depth=2, random_state=0)
    classifier.fit(X_train, y_train)  
    y_predict = classifier.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
#     print(cm)
    return [accuracy_score(y_test, y_predict)*100,
            f1_score(y_test, y_predict, average='weighted')]

############### All Emotions ####################
def RandomForestCombined(UserName,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'ValenceAndArousalEncoded'+".csv")
    SurveyFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
# Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)  	
# LDA Classifier Valence=y
    lda = LDA(n_components=15)  
    X_train = lda.fit_transform(X_train, y_train)  
    X_test = lda.transform(X_test)  
    classifier = RandomForestClassifier(max_depth=10, random_state=0)
    classifier.fit(X_train, y_train)  
    y_predict = classifier.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
#     print(cm)
    return [accuracy_score(y_test, y_predict)*100,
            f1_score(y_test, y_predict, average='weighted')]
    
def Run100(UserName,Window=256,StartTime=0,EndTime=63):
    arousal=[]
    valence=[]
    F1_ScoreArousal=[]
    F1_ScoreValence=[]
    for i in range (100):
        RandomForest=RandomForestLDA(UserName,Window,StartTime,EndTime)
        arousal+=[RandomForest[0]]
        valence+=[RandomForest[1]]
        F1_ScoreArousal+=[RandomForest[2]]
        F1_ScoreValence+=[RandomForest[3]]
    print("Arousal "+str(np.mean(arousal)))
    print(collections.Counter(arousal))
    print("Valence "+str(np.mean(valence)))
    print(collections.Counter(valence))
    return [np.mean(arousal),np.mean(valence),np.mean(F1_ScoreArousal),np.mean(F1_ScoreValence)];
    

    
def RunCombined100(UserName,Window=256,StartTime=0,EndTime=63):
    Combined=[]
    F1_ALL=[]
    for i in range (100):
        AllEmotions=RandomForestCombined(UserName,Window,StartTime,EndTime)
        Combined+=[AllEmotions[0]]
        F1_ALL+=[AllEmotions[1]]
    print("Array of average All Emotions accurecies for user "+UserName)
    print(collections.Counter(Combined))
    return [np.mean(Combined),np.mean(F1_ALL)];
    
def RunHS100(UserName,Window=256,StartTime=0,EndTime=63):
    Combined=[]
    F1_HS=[]
    for i in range (100):
        HS=RandomForestHS(UserName,Window,StartTime,EndTime)
        Combined+=[HS[0]]
        F1_HS+=[HS[1]]
    print("Array of average Happy Sad accurecies for user "+UserName)
    print(collections.Counter(Combined))
    return [np.mean(Combined),np.mean(F1_HS)];

if __name__ == "__main__":
    HSAccurecy=0;
    HSF1=0;
    CombinedAccurecy=0;
    CombinedF1=0;
    ValenceAccurecy=0;
    ValenceF1=0;
    ArousalAccurecy=0;
    ArousalF1=0;
    for i in range(1,10):
        print("s0"+str(i))
        #Happy and Sad
        var =RunHS100("s0"+str(i))
        print("Happy Sad")
        print(var)
        HSAccurecy+=var[0]
        HSF1+=var[1]
        #All 4 Emotions
        var1 =RunCombined100("s0"+str(i))
        print("Combined Emotions")
        print(var1)
        CombinedAccurecy+=var1[0]
        CombinedF1+=var1[1]
        #Seperated Valence and Arousal
        var2=Run100("s0"+str(i))
        print("Arousal Valence")
        print(var2)
        ArousalAccurecy+=var2[0]
        ValenceAccurecy+=var2[1]
        ArousalF1+=var2[2]
        ValenceF1+=var2[3]
    for i in range(10,31):
        print("s"+str(i))
        #Happy and Sad
        var =RunHS100("s"+str(i))
        print("Happy Sad")
        print(var)
        HSAccurecy+=var[0]
        HSF1+=var[1]
        #All 4 Emotions
        var1 =RunCombined100("s"+str(i))
        print("Combined Emotions")
        print(var1)
        CombinedAccurecy+=var1[0]
        CombinedF1+=var1[1]
        #Seperated Valence and Arousal
        var2=Run100("s"+str(i))
        print("Arousal Valence")
        print(var2)
        ArousalAccurecy+=var2[0]
        ValenceAccurecy+=var2[1]
        ArousalF1+=var2[2]
    print("Accurecy happy sad for first 30 "+str(HSAccurecy/30))
    print("F1 happy sad for first 30 "+str(HSF1/30))
    print("Accurecy Combined sad for first 30 "+str(CombinedAccurecy/30))
    print("F1 Combined sad for first 30 "+str(CombinedF1/30))
    print("Accurecy Valence for first 30 "+str(ValenceAccurecy/30))
    print("F1 Valence for first 30 "+str(ValenceF1/30))
    print("Accurecy Arousal for first 30 "+str(ArousalAccurecy/30))
    print("F1 Arousal for first 30 "+str(ArousalF1/30))



        

s01
Array of average Happy Sad accurecies for user s01
Counter({60.0: 36, 40.0: 26, 80.0: 25, 20.0: 7, 100.0: 6})
Happy Sad
[59.4, 0.5735873015873014]
Array of average All Emotions accurecies for user s01
Counter({37.5: 32, 25.0: 31, 50.0: 12, 12.5: 11, 62.5: 9, 0.0: 5})
Combined Emotions
[32.75, 0.3228115079365079]
Arousal 51.625
Counter({50.0: 31, 62.5: 21, 37.5: 19, 75.0: 12, 25.0: 8, 87.5: 4, 12.5: 3, 100.0: 1, 0.0: 1})
Valence 47.875
Counter({37.5: 33, 50.0: 22, 62.5: 20, 75.0: 11, 25.0: 10, 12.5: 2, 100.0: 1, 0.0: 1})
Arousal Valence
[51.625, 47.875, 0.48521828171828174, 0.46743578643578654]
s02
Array of average Happy Sad accurecies for user s02
Counter({60.0: 43, 80.0: 32, 40.0: 13, 20.0: 6, 100.0: 6})
Happy Sad
[63.8, 0.6566904761904762]
Array of average All Emotions accurecies for user s02
Counter({37.5: 37, 25.0: 20, 50.0: 17, 62.5: 14, 12.5: 9, 75.0: 2, 100.0: 1})
Combined Emotions
[39.75, 0.37791747835497835]
Arousal 60.375
Counter({62.5: 30, 50.0: 27, 75.0: 15, 87.5: 14, 3

Arousal 56.5
Counter({62.5: 29, 75.0: 23, 50.0: 21, 37.5: 16, 25.0: 8, 87.5: 3})
Valence 62.625
Counter({75.0: 30, 62.5: 24, 50.0: 23, 37.5: 12, 87.5: 9, 100.0: 1, 25.0: 1})
Arousal Valence
[56.5, 62.625, 0.5229244366744367, 0.6220714285714286]
s15
Array of average Happy Sad accurecies for user s15
Counter({50.0: 40, 75.0: 36, 25.0: 17, 100.0: 5, 0.0: 2})
Happy Sad
[56.25, 0.548952380952381]
Array of average All Emotions accurecies for user s15
Counter({37.5: 33, 25.0: 25, 12.5: 18, 50.0: 13, 0.0: 5, 62.5: 5, 75.0: 1})
Combined Emotions
[31.25, 0.2880535714285714]
Arousal 48.75
Counter({50.0: 26, 62.5: 24, 37.5: 19, 25.0: 12, 75.0: 11, 12.5: 5, 87.5: 2, 0.0: 1})
Valence 59.625
Counter({62.5: 24, 75.0: 24, 50.0: 23, 37.5: 11, 87.5: 6, 25.0: 6, 100.0: 4, 12.5: 2})
Arousal Valence
[48.75, 59.625, 0.45835997335997336, 0.5923586691086691]
s16
